## Original Dataset with Real Oil Prices

In [1]:
import pandas as pd

# read the Excel file, sheet name is 'Data 1'
rwtc_data = pd.read_excel('../Original/RWTCm.xls', sheet_name='Data 1', skiprows=2)

# print the column names of the RWTC data
print(rwtc_data.columns)

# Convert 'Date' in RWTC data from 'Jan-1986' to a datetime object, assuming the day is the first of the month
rwtc_data['Date'] = pd.to_datetime(rwtc_data['Date'], format='%b-%Y').dt.to_period('M')

# read the macro data
macro_data = pd.read_csv('../Original/Macroeconomic Data.csv')

# convert the date column in macro data from '1/1/1986' to a datetime object
macro_data['date'] = pd.to_datetime(macro_data['date'], format='%m/%d/%Y').dt.to_period('M')

# rename rwtc 'Date' column to 'date' to match the macro_data
rwtc_data.rename(columns={'Date': 'date'}, inplace=True)

# combine the datasets on the 'date' column
# how='left' means that we keep all the rows from the left df
combined_data = pd.merge(macro_data, rwtc_data, on='date', how='left')

# check the combined dataset
print(combined_data.head())

# Save the combined dataset to a new CSV file
combined_data.to_csv('./CSV/Combined_Raw.csv', index=False)

Index(['Date', 'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)'], dtype='object')
      date       RPI  W875RX1  DPCERA3M086SBEA   CMRMTSPLx      RETAILx  \
0  1986-01  6748.379   5932.0           47.357  665846.179  122987.5021   
1  1986-02  6781.198   5962.7           47.240  662058.599  121841.4787   
2  1986-03  6845.164   6018.8           47.401  662704.671  120996.2216   
3  1986-04  6868.394   6038.8           47.693  676048.783  121767.8428   
4  1986-05  6876.660   6043.6           47.980  669565.217  123536.1414   

    INDPRO  IPFPNSS  IPFINAL  IPCONGD  ...  DSERRG3M086SBEA  CES0600000008  \
0  55.6511  63.3746  61.7948  72.9945  ...           45.557          10.13   
1  55.2700  62.7722  61.1830  72.3652  ...           45.768          10.14   
2  54.8828  62.5615  60.9771  72.1034  ...           45.968          10.15   
3  54.9456  62.7816  61.0642  72.5823  ...           46.051          10.16   
4  55.0535  62.9988  61.2052  72.9877  ...           46.110          10.

### Apply Real Oil Price Conversion with Inflation (CPI)

In [2]:
# calculate the CPI adjusted oil price
combined_data['Real Oil Prices'] = (combined_data['Cushing, OK WTI Spot Price FOB (Dollars per Barrel)'] / combined_data['CPIAUCSL']) * 100

# check the results
print(combined_data[['Cushing, OK WTI Spot Price FOB (Dollars per Barrel)', 'CPIAUCSL', 'Real Oil Prices']].head())

# drop the Crushing, OK WTI Spot Price FOB (Dollars per Barrel) column
combined_data.drop(columns=['Cushing, OK WTI Spot Price FOB (Dollars per Barrel)'], inplace=True)


# save the updated dataset to a new CSV file
combined_data.to_csv('./CSV/Combined_CPI_Adjusted.csv', index=False)

   Cushing, OK WTI Spot Price FOB (Dollars per Barrel)  CPIAUCSL  \
0                                              22.93       109.9   
1                                              15.46       109.7   
2                                              12.61       109.1   
3                                              12.84       108.7   
4                                              15.38       109.0   

   Real Oil Prices  
0        20.864422  
1        14.092981  
2        11.558203  
3        11.812328  
4        14.110092  


In [3]:
combined_data.describe()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx,Real Oil Prices
count,450.000000,450.000000,450.000000,4.490000e+02,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,...,450.000000,450.000000,450.000000,450.000000,450.000000,449.000000,449.000000,450.000000,450.000000,450.000000
mean,12019.738071,10124.240222,85.772889,1.136712e+06,328720.343813,86.311223,91.696498,90.601448,96.233411,86.492993,...,84.559478,17.651089,20.182556,16.273222,86.624889,206531.987617,549736.451381,2015.393958,20.392776,22.485394
std,3392.675497,2591.297290,23.853921,2.702338e+05,144298.855450,15.569268,13.253291,13.739174,10.094681,17.554943,...,24.206585,5.136002,5.900577,4.404558,12.735462,105549.385831,264114.130970,1370.463138,8.002081,10.897338
min,6748.379000,5932.000000,47.240000,6.620586e+05,120996.221600,54.837000,62.561500,60.953500,72.103400,51.857200,...,45.557000,10.130000,11.730000,9.530000,50.000000,58381.000000,121947.590000,450.100900,10.078500,6.462521
25%,8735.766250,7499.050000,62.484250,8.810833e+05,203469.500000,71.460525,80.139625,78.230700,90.397175,73.386750,...,64.149500,12.902500,14.702500,12.287500,77.600000,91622.620000,260669.110000,896.071025,14.446025,14.190002
50%,11961.318000,10272.250000,89.221000,1.162401e+06,325654.500000,91.895000,98.130250,96.894600,99.457200,92.597250,...,82.563000,17.305000,19.230000,16.285000,89.900000,237064.040000,687869.270000,1721.780700,18.749700,18.585863
75%,14341.578250,11922.700000,102.484500,1.339979e+06,422314.500000,99.118200,100.600125,100.752550,102.548800,101.562250,...,103.843750,21.477500,24.477500,19.482500,95.250000,302075.410000,757762.530000,2753.991725,24.303375,29.227620
max,21448.063000,14754.400000,130.878000,1.585920e+06,692501.000000,104.118100,110.221900,108.677700,111.785000,110.128800,...,136.989000,29.550000,34.090000,26.410000,112.000000,395393.700000,896985.060000,5836.411700,62.964800,61.564496


### Count Missing Values (blanks)

In [4]:
missing_values_count = combined_data.isnull().sum()
total_missing_values = missing_values_count.sum()
print(f'Total missing values: {total_missing_values}')
print(missing_values_count) 

Total missing values: 90
date               0
RPI                0
W875RX1            0
DPCERA3M086SBEA    0
CMRMTSPLx          1
                  ..
DTCOLNVHFNM        1
DTCTHFNM           1
INVEST             0
VIXCLSx            0
Real Oil Prices    0
Length: 128, dtype: int64


### Count Zeros

In [5]:
zero_values_count = (combined_data == 0).astype(int).sum(axis=0)
total_zero_values = zero_values_count.sum()
print(f'Total zero values: {total_zero_values}')
print(zero_values_count)

Total zero values: 46
date               0
RPI                0
W875RX1            0
DPCERA3M086SBEA    0
CMRMTSPLx          0
                  ..
DTCOLNVHFNM        0
DTCTHFNM           0
INVEST             0
VIXCLSx            0
Real Oil Prices    0
Length: 128, dtype: int64


### Count Negatives

In [6]:
# Select only the numeric columns from DataFrame
numeric_df = combined_data.select_dtypes(include=['number'])

negative_values_count = (numeric_df < 0).sum()
total_negative_values = negative_values_count.sum()
print(f'Total negative values: {total_negative_values}')
print(negative_values_count)

Total negative values: 1022
RPI                0
W875RX1            0
DPCERA3M086SBEA    0
CMRMTSPLx          0
RETAILx            0
                  ..
DTCOLNVHFNM        0
DTCTHFNM           0
INVEST             0
VIXCLSx            0
Real Oil Prices    0
Length: 127, dtype: int64
